# Comparing rainfall across cities
##### Author: John Mays

In [28]:
import pandas as pd
from city_comparison_helpers import *

The question here is, what cities should I consider?  The more cities I consider, the more data I'd have to download.  If I downloaded the **GSOY** CSV for every one of the 83,000 or so weather stations that are listed, I'd have to download ~16GB of CSVs (a rough and likely low estimate).  If I wanted to bring in some big data technologies, I could do it, but I don't. So, should I...
- get a list of cities commonly considered rainy and find one weather station per city?
- get one weather station for every "big" city in the U.S.
- get all of the weather stations in a few states?

Although it's not perfect, I will likely go with the first option

## List of U.S. cities and towns commonly considered rainy:
- Seattle, WA
- Miami, FL
- New Orleans, LA
- Birmingham, AL
- Tampa, FL
- NY, NY
- White Plains, NY
- Brookhaven, NY
- Syracuse, NY
- Buffalo, NY
- Erie, PA
- San Francisco, CA
- Rochester, NY
- Cleveland, OH
- Akron, OH
- Pittsburgh, OR
- Portland, OR
- Salem, OR
- Houston, TX
- Ft. Lauderdale, FL
- Tallahasse, FL
- Aberdeen, WA
- Detroit, MI
- Forks, WA

**ANYWHERE BUT AKRON, OH!!:**

To be honest, I am mostly only interested in living North of the Mason-Dixon line, and also not particularly interested in living in the "middle of nowhere" so I'll take cities I am not interested in off of the list.

I'll also add a few cities I'm just curious about to come up with this list:

In [26]:
cities_stations = {
    "Seattle": "USW00094290",
    "Miami": "USW00092811", # cuts off at 2021 (Dec)
    "New York City": "USW00094728",
    "White Plains": "USW00094745", # techinically Westchester County 2011,13,14
    "Brookhaven": "USW00054790",
    "Syracuse": "USW00014771",
    "Buffalo": "USW00014733",
    "Erie": "USW00014860",
    "Pittsburgh": "USW00014762",
    "San Francisco": "USW00023272",
    "West Lafayette": "USC00129435", # MISSING YEARS 2021
    # "Rochester": "USW0001476",
    "Cleveland": "USW00014820",
    "Portland": "USC00356750",
    "Detroit": "USW00014822",
    "Forks": "USC00452914",
    "Boston": "USW00014739",
    "Madison": "USC00470273",
    "Woods Hole": "US1MABA0013", # technically Falmouth, MA  ## MISSING YEARS (tons)
    "Monterey": "USW00023259",
    "Richland": "USC00457015" ## MISSING 13,14
}

In [3]:
def total_concat(city_dict):
    '''
        Intelligently concatenates the individual dataframes for all of
        the cities
        @param
    '''
    pass

In [4]:
city_one = pd.read_csv("./data/GSOY/US1MABA0013.csv")
city_two = pd.read_csv("./data/GSOY/US1WAKG0005.csv")

In [9]:
print(list(city_one.columns))
print(list(city_two.columns))

['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'DP01', 'DP01_ATTRIBUTES', 'DP10', 'DP10_ATTRIBUTES', 'DP1X', 'DP1X_ATTRIBUTES', 'EMXP', 'EMXP_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES']
['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'DP01', 'DP01_ATTRIBUTES', 'DP10', 'DP10_ATTRIBUTES', 'DP1X', 'DP1X_ATTRIBUTES', 'DSND', 'DSND_ATTRIBUTES', 'DSNW', 'DSNW_ATTRIBUTES', 'EMSD', 'EMSD_ATTRIBUTES', 'EMSN', 'EMSN_ATTRIBUTES', 'EMXP', 'EMXP_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES']


In [14]:
print(city_one.shape)
print(city_two.shape)

(6, 16)
(11, 26)


- `SNOW` in city_two but not in city_one
- 

# YEAR METHOD TESTING:

In [12]:
cities_stations['Bellevue']

'US1WAKG0005'

In [13]:
city = pd.read_csv("./data/GSOY/US1WAKG0005.csv")

In [16]:
list(city["DATE"])

[2009, 2010, 2012, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [27]:
check_years(cities_stations)

Bellevue is missing years:
2013
2014
Miami is missing years:
2018
2019
2020
West Lafayette is missing years:
2021
Detroit is missing years:
2014
Woods Hole is missing years:
2013
2015
2016
2018
2021
Richland is missing years:
2014
2015


In [17]:
for i in range(2012,2021):
    print(i)

2012
2013
2014
2015
2016
2017
2018
2019
2020


In [19]:
list(cities_stations.keys())

['Bellevue',
 'Seattle',
 'Miami',
 'New York City',
 'White Plains',
 'Brookhaven',
 'Syracuse',
 'Buffalo',
 'Erie',
 'Pittsburgh',
 'San Francisco',
 'West Lafayette',
 'Rochester',
 'Cleveland',
 'Portland',
 'Detroit',
 'Forks',
 'Boston',
 'Madison',
 'Woods Hole',
 'Monterey',
 'Richland']